In [123]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys
sys.path.append('../../')
import numpy as np
from pyrl.tasks.pacman.game_mdp import *
from pyrl.tasks.pacman.ghostAgents import *
import pyrl.tasks.pacman.graphicsDisplay as graphicsDisplay
import pyrl.tasks.pacman.textDisplay as textDisplay
from pyrl.utils import Timer
from pyrl.prob import choice
import pyrl.agents.arch as arch
from pyrl.algorithms.valueiter import DeepQlearn
from pyrl.algorithms.curriculum import *
from pyrl.agents.agent import DQN
from pyrl.evaluate import reward_stochastic
import theano
import theano.tensor as T
import json

## Test Components

In [124]:
_layout = layout.getLayout('pyrl/tasks/pacman/layouts/smallGrid.lay')
ghostType = DirectionalGhost
agents = [ghostType( i+1 ) for i in range(1)]
#display = graphicsDisplay.PacmanGraphics(zoom=1.0, frameTime = 0.1)
# display = textDisplay.PacmanGraphics()
display = textDisplay.NullGraphics()
task = PacmanTask(_layout, agents, display, state_repr='stack')

In [20]:
# pacman task feature.
feat_func = PacmanTaskFeature.featurize_num_ghost
feat_func(task)
print 'feat', feat

feat {'#ghost': 1.0, 'bias': 1.0}


In [12]:
# meta model tabular
meta_model = MetaModelTabular(bonus=1., decay=0.7)
meta_model.learn(feat, 3)

In [13]:
meta_model.get(feat)

1.9000000000000001

In [16]:
# edit function.
edit_func = lambda task: PacmanTaskShifter.neighbors(task, axis=['del_ghost', 'add_ghost'])

In [17]:
task_nb = edit_func(task)

In [19]:
print task_nb[1]

%%%%%%%
% <   %
% %%% %
% %.  %
%G%%% %
%. G  %
%%%%%%%
Score: 0



In [25]:
feat_nb = feat_func(task_nb[1])
print 'feat', feat_nb
meta_model.learn(feat_nb, 3)
meta_model.get(feat_nb)

feat {'#ghost': 2.0, 'bias': 1.0}


2.5300000000000002

In [26]:
meta_model.get(feat)

1.9000000000000001

## Test Basic Learning

In [125]:
input_dim = task.state_shape

def two_layer(states):
    state_vecs = states.flatten(2)
    return arch.two_layer(state_vecs, np.prod(task.state_shape), 128, task.num_actions)
    
_layout = layout.getLayout('pyrl/tasks/pacman/layouts/smallGrid.lay')
ghostType = DirectionalGhost
agents = [ghostType( i+1 ) for i in range(1)]
#display = graphicsDisplay.PacmanGraphics(zoom=1.0, frameTime = 0.1)
# display = textDisplay.PacmanGraphics()
display = textDisplay.NullGraphics()
task = PacmanTask(_layout, agents, display, state_repr='stack')

In [126]:
dqn = DQN(task, arch_func=two_layer, state_type=T.tensor4)
feat_func = PacmanTaskFeature.featurize_num_ghost
edit_func = lambda task: PacmanTaskShifter.neighbors(task, axis=['del_ghost', 'add_ghost'])
meta_model = MetaModelTabular(bonus=100., decay=0.4)

In [127]:
learner = DQCL_LocalEdit(dqn, edit_func, feat_func, meta_model, gamma=0.95, lr=1e-3, memory_size = 100, epsilon=0.5)

In [129]:
learner.run(task, num_epochs=30, num_episodes=30, tol=1e-4)

val 28.5238333332
val_nb [100.0, 121.03841629366713, 48.544690727404785]
probability [  7.29679080e-10   9.99999999e-01   3.28378726e-32]
new_task %%%%%%%
% <   %
% %%% %
% %.  %
%G%%% %
%. G  %
%%%%%%%
Score: 0

val 21.3280195369
val_nb [48.544690727404785, 48.544690727404785, 100.0, 61.212178239611859]
probability [  4.50031712e-23   4.50031712e-23   1.00000000e+00   1.42778575e-17]
new_task %%%%%%%
% <   %
% %%% %
%G%.  %
%G%%% %
%. G  %
%%%%%%%
Score: 0

val 20.0434801539
val_nb [61.212178239611859, 61.212178239611859, 61.212178239611859, 100.0, 52.02608809231144]
probability [  1.42778575e-17   1.42778575e-17   1.42778575e-17   1.00000000e+00
   1.46283312e-21]
new_task %%%%%%%
% <   %
%G%%% %
%G%.  %
%G%%% %
%. G  %
%%%%%%%
Score: 0

val 15.4024402007
val_nb [52.02608809231144, 52.02608809231144, 52.02608809231144, 52.02608809231144, 100.0, 49.241464120394355]
probability [  1.46283312e-21   1.46283312e-21   1.46283312e-21   1.46283312e-21
   1.00000000e+00   9.03333171e-23]
new_

Exception: Illegal ghost action North

In [131]:
print learner.meta_model.model

{'{"#ghost": 15.0, "bias": 1.0}': 61.204225068267725, '{"#ghost": 2.0, "bias": 1.0}': 61.212178239611859, '{"#ghost": 14.0, "bias": 1.0}': 75.05823370787904, '{"#ghost": 4.0, "bias": 1.0}': 49.241464120394355, '{"#ghost": 1.0, "bias": 1.0}': 48.544690727404785, '{"#ghost": 16.0, "bias": 1.0}': 51.486576740433392, '{"#ghost": 3.0, "bias": 1.0}': 52.02608809231144, '{"#ghost": 11.0, "bias": 1.0}': 72.527845871138695, '{"#ghost": 12.0, "bias": 1.0}': 72.594571851028064, '{"#ghost": 7.0, "bias": 1.0}': 73.978305836533494, '{"#ghost": 6.0, "bias": 1.0}': 73.1922179229243, '{"#ghost": 13.0, "bias": 1.0}': 86.954514248743706, '{"#ghost": 8.0, "bias": 1.0}': 72.470428561766624, '{"#ghost": 5.0, "bias": 1.0}': 57.322797756705384, '{"#ghost": 10.0, "bias": 1.0}': 72.482812377889246, '{"#ghost": 9.0, "bias": 1.0}': 72.407389232083716}
